In [14]:
import boto3
import time
from botocore.exceptions import ClientError, WaiterError

# --------- CONFIG ---------
region = "ap-south-1"  # Mumbai region
ami_id = "ami-0d0ad8bXXXXXb745"  # Replace if required
instance_type = "t2.micro"
key_name = "project_2"  # Your key pair name
security_group_id = "sg-0b4772XXXXX78b7"  # Your security group ID

# Create EC2 client
ec2 = boto3.client("ec2", region_name=region)

# --------- LAUNCH EC2 ---------
def launch_instance():
    print("Launching EC2 instance...")
    try:
        response = ec2.run_instances(
            ImageId=ami_id,
            InstanceType=instance_type,
            KeyName=key_name,
            MaxCount=1,
            MinCount=1,
            SecurityGroupIds=[security_group_id],
            TagSpecifications=[
                {
                    "ResourceType": "instance",
                    "Tags": [{"Key": "Name", "Value": "MyInstance"}]
                }
            ]
        )
    except ClientError as e:
        print(f"Error launching instance: {e}")
        return None

    instance_id = response['Instances'][0]['InstanceId']
    print(f"Instance launched with ID: {instance_id}")

    print("Waiting for instance to be in 'running' state...")
    waiter = ec2.get_waiter('instance_running')
    waiter.wait(InstanceIds=[instance_id])
    print("Instance is running.")

    # Get public IP
    public_ip = get_public_ip(instance_id)
    if public_ip:
        print(f"Public IP: {public_ip}")

    return instance_id

# Get Public IP
def get_public_ip(instance_id):
    try:
        desc = ec2.describe_instances(InstanceIds=[instance_id])
        return desc['Reservations'][0]['Instances'][0].get('PublicIpAddress')
    except ClientError:
        return None

# --------- TERMINATE EC2 ---------
def terminate_instance(instance_id):
    print(f"Terminating instance {instance_id}...")
    try:
        ec2.terminate_instances(InstanceIds=[instance_id])
        print("Waiting for instance to be terminated...")
        waiter = ec2.get_waiter('instance_terminated')
        waiter.wait(InstanceIds=[instance_id])
    except (ClientError, WaiterError) as e:
        print(f"Error terminating instance: {e}")
    print("Instance terminated.")

In [10]:
instance_id = launch_instance()

Launching EC2 instance...
Instance launched with ID: i-03edb97910e3f78db
Waiting for instance to be in 'running' state...
Instance is running.
Public IP: 65.1.147.250


In [11]:
terminate_instance(instance_id)

Terminating instance i-03edb97910e3f78db...
Waiting for instance to be terminated...
Instance terminated.


In [12]:
instance_id = launch_instance()

Launching EC2 instance...
Instance launched with ID: i-0371084f55b4df88b
Waiting for instance to be in 'running' state...
Instance is running.
Public IP: 13.233.36.42


In [13]:
terminate_instance(instance_id)

Terminating instance i-0371084f55b4df88b...
Waiting for instance to be terminated...
Instance terminated.
